In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def kpss_veri_cek(yil: int, donem: int, max_sayfa: int = 38):
    """
    Belirtilen yıl ve dönemdeki KPSS atama verilerini memurlar.net'ten çeker.

    Args:
        yil (int): Verilerin çekileceği yıl (örn: 2024).
        donem (int): Atama dönemi (1 veya 2).
        max_sayfa (int): Taranacak maksimum sayfa sayısı.

    Returns:
        pandas.DataFrame: Çekilen verileri içeren DataFrame.
    """
    
    # Her sütun için boş listeler oluşturulur
    program_kodlari = []
    kurum_adlari = []
    kadro_unvanlari = []
    iller = []
    kontenjanlar = []
    nitelikler_listesi = []
    bos_kontenjanlar = []
    min_puanlar = []
    max_puanlar = []

    print(f"--- {yil}/{donem} KPSS verileri çekilmeye başlanıyor ---")

    for sayfa in range(1, max_sayfa + 1):
        # URL, fonksiyona verilen yıl ve dönem parametreleri ile dinamik olarak oluşturulur
        URL = f'https://kpss.memurlar.net/kpssrobotu/{yil}/{donem}/{sayfa}.sayfa?SelectedItem=Kadro+Arama&BranchType={ogrenim}&BranchCode=&Organization=&JobTitle=&CityCode='

        print(f"{sayfa}. sayfa taranıyor: {URL}")
        
        try:
            response = requests.get(URL, timeout=10) # Zaman aşımı eklemek iyi bir pratiktir
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Tablodaki satırları seç
            rows = soup.select('table.KpssList tr')[1:]
            
            # Eğer sayfa boşsa veya veri yoksa döngüyü sonlandır
            if not rows:
                print(f"{sayfa}. sayfada veri bulunamadı. Veri çekme işlemi tamamlandı.")
                break

            for row in rows:
                cols = row.find_all('td')
                if len(cols) < 10:
                    continue
                
                text = cols[2].text.strip()

                pattern = re.compile(r'\d+')

                match = pattern.search(text)

                if match:
                    split_index = match.start()
    
                oncesi = text[:split_index]
                sonrasi = text[split_index:]

                program_kodlari.append(cols[1].text.strip())
                kurum_adlari.append(oncesi)
                nitelikler_listesi.append(sonrasi)
                kadro_unvanlari.append(cols[len(cols)-6].text.strip())
                iller.append(cols[len(cols)-5].text.strip())
                kontenjanlar.append(cols[len(cols)-4].text.strip())
                bos_kontenjanlar.append(cols[len(cols)-3].text.strip())
                min_puanlar.append(cols[len(cols)-2].text.strip())
                max_puanlar.append(cols[len(cols)-1].text.strip())

        except requests.exceptions.RequestException as e:
            print(f"Hata oluştu: {e}. Bu sayfa atlanıyor.")
            continue
    
    # Toplanan listeler kullanılarak DataFrame oluşturulur
    df = pd.DataFrame({
        'Program Kodu': program_kodlari,
        'Kurum Adı': kurum_adlari,
        'Ünvanı': kadro_unvanlari,
        'Kadro Detayları': nitelikler_listesi,
        'İl': iller,
        'Kontenjan': kontenjanlar,
        'Boş Kontenjan': bos_kontenjanlar,
        'Min Puan': min_puanlar,
        'Max Puan': max_puanlar,
    })
    
    print(f"\nToplam {len(df)} adet kadro bilgisi çekildi.")
    return df

# --- KODU KULLANMA ÖRNEĞİ ---

# Örnek 1: 2024 yılı 2. atama verilerini çekmek için
# yil = 2024
# donem = 2
# kpss_df = kpss_veri_cek(yil, donem)


# Örnek 2: 2025 yılı 1. atama verilerini çekmek için
yil = 2025
donem = 3
ogrenim = 1 # 1 ortaöğretim - 2 lise - 3 üniversite
kpss_df = kpss_veri_cek(yil, donem)


# DataFrame'in boş olup olmadığını kontrol et
if not kpss_df.empty:
    # Verileri ekrana yazdırma
    pd.set_option('display.max_rows', 100) # Ekrana 100 satır yazdır
    print("\n--- Çekilen Verilerin Önizlemesi ---")
    print(kpss_df.head()) # İlk 5 satırı göster

# Verileri bir CSV dosyasına kaydetme
# Dosya adının başına /tmp/ ekleyerek yazılabilir bir konuma kaydediyoruz.
    dosya_adi = f'/Users/gp/Desktop/kpss-veri-cekme/kpss_{yil}_{donem}_kadrolari.csv' 
    kpss_df.to_csv(dosya_adi, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{dosya_adi}' dosyasına kaydedildi.")  
else:
    print("\nİlgili dönem için herhangi bir veri çekilemedi.")

Son sayfa numarası tespit ediliyor: https://kpss.memurlar.net/kpssrobotu/2024/2/1.sayfa?SelectedItem=Kadro+Arama&BranchType=4
Tespit edilen son sayfa numarası: 13
--- 2024/2 KPSS verileri çekilmeye başlanıyor (13 sayfa taranacak) ---
1. sayfa taranıyor...
2. sayfa taranıyor...
3. sayfa taranıyor...
4. sayfa taranıyor...
5. sayfa taranıyor...
6. sayfa taranıyor...
7. sayfa taranıyor...
8. sayfa taranıyor...
9. sayfa taranıyor...
10. sayfa taranıyor...
11. sayfa taranıyor...
12. sayfa taranıyor...
13. sayfa taranıyor...

Toplam 560 adet kadro bilgisi çekildi.

--- Çekilen Verilerin Önizlemesi ---
  Program Kodu                                          Kurum Adı  \
0    302010920              Eti Maden İşletmeleri Genel Müdürlüğü   
1    302011277                         Hazine ve Maliye Bakanlığı   
2    302013020      Türkiye Elektrik Dağıtım A.Ş. Genel Müdürlüğü   
3    302011781  T.C.Devlet Demiryolları İşletmesi Genel Müdürlüğü   
4    302012999      Türkiye Elektrik Dağıtım A.Ş. Gen

In [58]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def get_son_sayfa_numarasi(yil: int, donem: int, ogrenim: int) -> int:
    """
    İlgili KPSS atama sayfasındaki en son sayfa numarasını, doğrudan 'Son Sayfa' 
    linkini hedefleyerek verimli bir şekilde bulur.
    """
    URL = f'https://kpss.memurlar.net/kpssrobotu/{yil}/{donem}/1.sayfa?SelectedItem=Kadro+Arama&BranchType={ogrenim}'
    print(f"Son sayfa numarası tespit ediliyor: {URL}")
    
    try:
        response = requests.get(URL, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # === İYİLEŞTİRME BURADA ===
        # Önceki gibi tüm linkleri taramak yerine, doğrudan class'ı 'Last' olan linki seçiyoruz.
        last_page_link = soup.select_one('a.Last')
        
        if last_page_link and 'href' in last_page_link.attrs:
            href = last_page_link['href']
            # Regex ile href içerisinden sayfa numarasını ayıklıyoruz. Örn: ".../38.sayfa?..." -> 38
            match = re.search(r'/(\d+)\.sayfa', href)
            if match:
                son_sayfa = int(match.group(1))
                print(f"Tespit edilen son sayfa numarası: {son_sayfa}")
                return son_sayfa
        
        # Eğer 'Last' linki bulunamazsa (tek sayfa varsa), en yüksek sayılı linki bulmayı deneriz.
        print("'Last' linki bulunamadı, alternatif yöntem deneniyor...")
        linkler = soup.find_all('a', string=re.compile(r'^\d+$'))
        if not linkler:
            print("Sayfalama bulunamadı, tek sayfa olduğu varsayılıyor.")
            return 1
        
        max_sayfa = max(int(link.text) for link in linkler)
        print(f"Tespit edilen son sayfa numarası (alternatif): {max_sayfa}")
        return max_sayfa

    except requests.exceptions.RequestException as e:
        print(f"Sayfa numarası alınırken hata oluştu: {e}. Varsayılan olarak 1 sayfa taranacak.")
        return 1


def kpss_veri_cek(yil: int, donem: int, ogrenim: int):
    """
    Belirtilen yıl ve dönemdeki KPSS atama verilerini memurlar.net'ten çeker.
    max_sayfa değerini otomatik olarak kendisi bulur.
    """
    
    max_sayfa = get_son_sayfa_numarasi(yil, donem, ogrenim)
    
    program_kodlari, kurum_adlari, kadro_unvanlari, iller, kontenjanlar, nitelikler_listesi, bos_kontenjanlar, min_puanlar, max_puanlar = ([] for i in range(9))

    print(f"--- {yil}/{donem} KPSS verileri çekilmeye başlanıyor ({max_sayfa} sayfa taranacak) ---")

    for sayfa in range(1, max_sayfa + 1):
        URL = f'https://kpss.memurlar.net/kpssrobotu/{yil}/{donem}/{sayfa}.sayfa?SelectedItem=Kadro+Arama&BranchType={ogrenim}'
        print(f"{sayfa}. sayfa taranıyor...")
        
        try:
            response = requests.get(URL, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            rows = soup.select('table.KpssList tr')[1:]
            
            if not rows:
                print(f"{sayfa}. sayfada veri bulunamadı.")
                continue

            for row in rows:
                cols = row.find_all('td')
                if len(cols) < 10: continue
                
                text = cols[2].text.strip()
                match = re.search(r'\d+', text)
                split_index = match.start() if match else len(text)
    
                oncesi = text[:split_index].strip()
                sonrasi = text[split_index:].strip()

                program_kodlari.append(cols[1].text.strip())
                kurum_adlari.append(oncesi)
                nitelikler_listesi.append(sonrasi)
                kadro_unvanlari.append(cols[len(cols)-6].text.strip())
                iller.append(cols[len(cols)-5].text.strip())
                kontenjanlar.append(cols[len(cols)-4].text.strip())
                bos_kontenjanlar.append(cols[len(cols)-3].text.strip())
                min_puanlar.append(cols[len(cols)-2].text.strip())
                max_puanlar.append(cols[len(cols)-1].text.strip())

        except requests.exceptions.RequestException as e:
            print(f"Hata oluştu: {e}. Bu sayfa atlanıyor.")
            continue
    
    df = pd.DataFrame({
        'Program Kodu': program_kodlari, 'Kurum Adı': kurum_adlari, 'Ünvanı': kadro_unvanlari,
        'Kadro Detayları': nitelikler_listesi, 'İl': iller, 'Kontenjan': kontenjanlar,
        'Boş Kontenjan': bos_kontenjanlar, 'Min Puan': min_puanlar, 'Max Puan': max_puanlar,
    })
    
    print(f"\nToplam {len(df)} adet kadro bilgisi çekildi.")
    return df

# --- KODU KULLANMA ---

# GERÇEKÇİ DEĞERLER: 2025/4 ataması olmadığı için, kodun çalışabilmesi adına 2024/2 atamaları kullanılmıştır.
yil = 2024
donem = 5
# Öğrenim Düzeyi: 1:Ortaöğretim, 2:Önlisans, 4:Lisans
ogrenim = 4
kpss_df = kpss_veri_cek(yil, donem, ogrenim)


if not kpss_df.empty:
    pd.set_option('display.max_rows', 100)
    print("\n--- Çekilen Verilerin Önizlemesi ---")
    print(kpss_df.head())

    # Not: 2: Ortaöğretim, 3: Önlisans gibi bir haritalama kullanmışsınız. 
    # Standartta genelde 2: Önlisans'tır. Kendi istediğiniz gibi bırakıyorum.
    ogrenim_str = {2: "ortaöğretim", 4: "lisans"}.get(ogrenim, "önlisans")

    # DOSYA YOLU DÜZELTMESİ: Hata almamak için geçici klasöre kaydedilir.
    dosya_adi = f'kpss_{yil}-{donem}_ogrenim_{ogrenim_str}_kadrolari.csv' 
    kayit_yolu = f'/Users/gp/Desktop/kpss-veri-cekme/{dosya_adi}' # Hata veren /Users/gp/... yolu yerine bu kullanılır.
    
    kpss_df.to_csv(kayit_yolu, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{kayit_yolu}' dosyasına kaydedildi.")  
else:
    print("\nİlgili dönem için herhangi bir veri çekilemedi.")

Son sayfa numarası tespit ediliyor: https://kpss.memurlar.net/kpssrobotu/2024/5/1.sayfa?SelectedItem=Kadro+Arama&BranchType=4
Tespit edilen son sayfa numarası: 52
--- 2024/5 KPSS verileri çekilmeye başlanıyor (52 sayfa taranacak) ---
1. sayfa taranıyor...
2. sayfa taranıyor...
3. sayfa taranıyor...
4. sayfa taranıyor...
5. sayfa taranıyor...
6. sayfa taranıyor...
7. sayfa taranıyor...
8. sayfa taranıyor...
9. sayfa taranıyor...
10. sayfa taranıyor...
11. sayfa taranıyor...
12. sayfa taranıyor...
13. sayfa taranıyor...
14. sayfa taranıyor...
15. sayfa taranıyor...
16. sayfa taranıyor...
17. sayfa taranıyor...
18. sayfa taranıyor...
19. sayfa taranıyor...
20. sayfa taranıyor...
21. sayfa taranıyor...
22. sayfa taranıyor...
23. sayfa taranıyor...
24. sayfa taranıyor...
25. sayfa taranıyor...
26. sayfa taranıyor...
27. sayfa taranıyor...
28. sayfa taranıyor...
29. sayfa taranıyor...
30. sayfa taranıyor...
31. sayfa taranıyor...
32. sayfa taranıyor...
33. sayfa taranıyor...
34. sayfa taranı

In [80]:
import csv
import json

# Giriş CSV dosyası
csv_file = '/Users/gp/Desktop/kpss-veri-cekme/kpss_2024-4_ogrenim_lisans_kadrolari.csv'
# Çıkış JSON dosyası
json_file = '/Users/gp/Desktop/kpss-veri-cekme/kpss_2024-4_ogrenim_lisans_kadrolari.json'

# CSV dosyasını oku
with open(csv_file, encoding='utf-8') as f:
    reader = csv.DictReader(f)
    rows = list(reader)

# JSON dosyasına yaz
with open(json_file, 'w', encoding='utf-8') as f:
    json.dump(rows, f, ensure_ascii=False, indent=4)

print("✅ CSV dosyası JSON'a başarıyla çevrildi!")


✅ CSV dosyası JSON'a başarıyla çevrildi!
